# Instalamos e importamos librerías

In [ ]:
from builtins import round as roundpy
from builtins import max as maxpy

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=dd8fa9017767b7423193252e38d749c3b24256cc7c42e24a5212731eef43cd8b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-upda

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Bajamos archivo con los datos del índice BigMac

In [ ]:
#https://drive.google.com/file/d/1y4_n4twjE4VSLSC-V7QsdPJ5JR9fiVgG/view?usp=sharing GooglePlayStore.csv
#https://drive.google.com/file/d/1HBgBzv4HU1wQKKblj8p--9lJEMSGYczg/view?usp=sharing GooglePlayStore_User_Reviews.csv

id1='1y4_n4twjE4VSLSC-V7QsdPJ5JR9fiVgG'
id2='1HBgBzv4HU1wQKKblj8p--9lJEMSGYczg'
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('GooglePlayStore.csv')
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('GooglePlayStore_User_Reviews.csv')

# Creamos el Spark Context

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Leemos CSV

In [ ]:
sqlContext = SQLContext(sc)
dfAppDetails = sqlContext.read.csv('GooglePlayStore.csv', header=True, inferSchema=True, mode='DROPMALFORMED')
dfAppReviews = sqlContext.read.csv('GooglePlayStore_User_Reviews.csv', header=True, inferSchema=True, mode='DROPMALFORMED')
rddAppDetails = dfAppDetails.rdd
rddAppReviews = dfAppReviews.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Preparación de Datasets

Droppeamos los apps completamente idénticos.

In [ ]:
rddAppDetails = rddAppDetails.distinct()

In [ ]:
rddAppDetails.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating='4.1', Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating='3.9', Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating='4.7', Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating='4.5', Reviews='215644', S

In [ ]:
rddAppReviews.take(5)

[Row(App='10 Best Foods for You', Translated_Review='This help eating healthy exercise regular basis', Sentiment='Positive', Sentiment_Polarity='0.25', Sentiment_Subjectivity='0.28846153846153844'),
 Row(App='10 Best Foods for You', Translated_Review='nan', Sentiment='nan', Sentiment_Polarity='nan', Sentiment_Subjectivity='nan'),
 Row(App='10 Best Foods for You', Translated_Review='Works great especially going grocery store', Sentiment='Positive', Sentiment_Polarity='0.4', Sentiment_Subjectivity='0.875'),
 Row(App='10 Best Foods for You', Translated_Review='Best idea us', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.3'),
 Row(App='10 Best Foods for You', Translated_Review='Best way', Sentiment='Positive', Sentiment_Polarity='1.0', Sentiment_Subjectivity='0.3')]

Descartamos los reviews sin Sentiment.

In [ ]:
rddAppReviews = rddAppReviews.filter(lambda x: x.Sentiment != 'nan')

# Ejercicio 9
Cuál es la app más cara de cada categoría?

Primero filtramos las aplicaciones pagas ya que sólo esas tienen precio, es argumentable que no debamos hacerlo teniendo en cuenta que pueden existir categorías sin aplicaciones pagas; en mi caso considero que no es necesario tenerlas en cuenta ya que no tendrían la "app más cara" y sólo suman cálculos innecesarios(más pensando que la proporción de aplicaciones gratis son mucho mayor a la de las pagas).


Luego, los mappeamos a (Categoría, Precio formatteado). Se decidió formatear los precios en Cent para poder manejarnos con enteros y no float.

In [ ]:
rddPaidAppPrice = rddAppDetails.filter(lambda x: x.Type == 'Paid').map(lambda x: (x.Category, (x.App, int(x.Price.lstrip('$').replace('.','')))))
rddPaidAppPrice.take(5)

[('BUSINESS', ('TurboScan: scan documents and receipts in PDF', 499)),
 ('BUSINESS', ('Tiny Scanner Pro: PDF Doc Scan', 499)),
 ('COMMUNICATION', ('Puffin Browser Pro', 399)),
 ('DATING', ('Moco+ - Chat, Meet People', 399)),
 ('DATING', ('Calculator', 699))]

Cacheo ya que lo voy a utilizar para count y collect(en caso que count dé un número acotado, que sabemos anticipadamente que sí si es que todo funcionó correctamente).

In [ ]:
mostExpensiveAppPerCategory = rddPaidAppPrice.reduceByKey(lambda a,b: (a[0], a[1]) if a[1]>b[1] else (b[0], b[1])).cache()
mostExpensiveAppPerCategory.count()

30

Uso collect() sabiendo que la cantidad de datos es acotada. La app más cara para cada categoría con los precios:

In [ ]:
mostExpensiveAppPerCategory.map(lambda x: (x[0], x[1][0], '$'+str(x[1][1]/100))).collect()

[('BUSINESS', 'Lean EQ', '$89.99'),
 ('COMMUNICATION', 'Z PIVOT', '$19.99'),
 ('DATING', 'AMBW Dating App: Asian Men Black Women Interracial', '$7.99'),
 ('EDUCATION', 'Fuzzy Numbers: Pre-K Number Foundation', '$5.99'),
 ('ENTERTAINMENT', 'My Talking Pet', '$4.99'),
 ('FOOD_AND_DRINK', 'Paprika Recipe Manager', '$4.99'),
 ('HEALTH_AND_FITNESS', 'Vital Tones EI Pro', '$9.99'),
 ('GAME', 'The World Ends With You', '$17.99'),
 ('FAMILY', 'I Am Rich Pro', '$399.99'),
 ('MEDICAL', 'EP Cook Book', '$200.0'),
 ('PHOTOGRAPHY', 'Guide to Nikon Df', '$29.99'),
 ('SPORTS', 'Golfshot Plus: Golf GPS', '$29.99'),
 ('PERSONALIZATION', 'BIG Launcher', '$9.99'),
 ('PRODUCTIVITY', 'cronometra-br', '$154.99'),
 ('WEATHER', 'Florida Tides & Weather', '$6.99'),
 ('TOOLS', 'G-NetReport Pro', '$25.99'),
 ('TRAVEL_AND_LOCAL', 'Maps & GPS Navigation OsmAnd+', '$8.99'),
 ('LIFESTYLE', "I'm Rich - Trump Edition", '$400.0'),
 ('AUTO_AND_VEHICLES', 'FORD V SERIES CALC - NO LIMIT', '$9.99'),
 ('NEWS_AND_MAGAZINES',